In [ ]:
!ls
from google.colab import drive
drive.mount('/content/drive')

sample_data


KeyboardInterrupt: ignored

In [ ]:
!wget http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz 

--2020-11-30 16:35:56--  http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100235015 (96M) [application/x-gzip]
Saving to: ‘parallel.tgz’

parallel.tgz        100%[===================>]  95.59M  2.60MB/s    in 55s     

2020-11-30 16:36:51 (1.75 MB/s) - ‘parallel.tgz’ saved [100235015/100235015]



In [ ]:
!tar -xzvf parallel.tgz
!ls

parallel/
parallel/IITB.en-hi.hi
parallel/IITB.en-hi.en
parallel  parallel.tgz	sample_data


In [ ]:
import pickle
from collections import defaultdict
import pandas as pd

cnt=0;
fe=open("parallel/IITB.en-hi.en","rb")
fh=open("parallel/IITB.en-hi.hi","rb")

d=defaultdict(list)
for x in fe.readlines():
	d['english'].append(x.decode('utf-8'))
	# d['hindi'].append(data['hindi'][ind])
	cnt+=1
	if cnt>=200000:
		break
cntt=0
for x in fh.readlines():
	d['hindi'].append(x.decode('utf-8'))
	cntt+=1
	if cntt>=200000:
		break
print(len(d['hindi']))
print(len(d['english']))
print(cntt,cnt)
data = pd.DataFrame(d)
print(data.shape)
cnt=0
filename="IITB_input.pkl"
outfile=open(filename,'wb')
pickle.dump(data,outfile)
outfile.close()
with open('IITB_input.pkl', 'rb') as f:
    dat = pickle.load(f)

print(type(dat))
cnt=0
for ind in dat.index: 
    print(dat['english'][ind], dat['hindi'][ind])
    cnt+=1
    if cnt>=10:
	    break 


!ls

200000
200000
200000 200000
(200000, 2)
<class 'pandas.core.frame.DataFrame'>
Give your application an accessibility workout
 अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें

Accerciser Accessibility Explorer
 एक्सेर्साइसर पहुंचनीयता अन्वेषक

The default plugin layout for the bottom panel
 निचले पटल के लिए डिफोल्ट प्लग-इन खाका

The default plugin layout for the top panel
 ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका

A list of plugins that are disabled by default
 उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है

Highlight duration
 अवधि को हाइलाइट रकें

The duration of the highlight box when selecting accessible nodes
 पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि

Highlight border color
 सीमांत (बोर्डर) के रंग को हाइलाइट करें

The color and opacity of the highlight border.
 हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। 

Highlight fill color
 भराई के रंग को हाइलाइट करें

IITB_input.pkl	parallel  parallel.tgz	sample_data


In [ ]:
import pickle

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm 


device = torch.device("cuda")

In [ ]:
from torchtext import data
from torchtext.data import Field, BucketIterator

import numpy as np


In [ ]:
!pip install indic-nlp-library

In [ ]:
import pandas as pd
df = pd.read_pickle("./IITB_input.pkl")
df.head(20)

,english,hindi
0,Give your application an accessibility workout\n,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें\n
1,Accerciser Accessibility Explorer\n,एक्सेर्साइसर पहुंचनीयता अन्वेषक\n
2,The default plugin layout for the bottom panel\n,निचले पटल के लिए डिफोल्ट प्लग-इन खाका\n
3,The default plugin layout for the top panel\n,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका\n
4,A list of plugins that are disabled by default\n,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
5,Highlight duration\n,अवधि को हाइलाइट रकें\n
6,The duration of the highlight box when selecti...,पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्स...
7,Highlight border color\n,सीमांत (बोर्डर) के रंग को हाइलाइट करें\n
8,The color and opacity of the highlight border.\n,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। \n
9,Highlight fill color\n,भराई के रंग को हाइलाइट करें\n


In [ ]:
import os
import random
import re
import string
from string import digits



from nltk.translate.bleu_score import sentence_bleu
lines = df.copy()
# general
lines['english']=lines['english'].apply(lambda x: re.sub(r"n\'t", " not", x))
lines['english']=lines['english'].apply(lambda x: re.sub(r"\'re", " are", x))
# lines['english']=lines['english'].apply(lambda x: re.sub(r"\'s", " is", x))
lines['english']=lines['english'].apply(lambda x: re.sub(r"n\'d", " would", x))
lines['english']=lines['english'].apply(lambda x: re.sub(r"n\'ll", " will", x))
lines['english']=lines['english'].apply(lambda x: re.sub(r"n\'ve", " have", x))
lines['english']=lines['english'].apply(lambda x: re.sub(r"n\'m", " am", x))

lines['hindi'] = lines['hindi'].apply(lambda x: x.replace('.',' ।'))
# lines['hindi'] = lines['hindi'].apply(lambda x: x.replace('।',' ।'))

# lines['english']=lines['english'].apply(lambda x: re.sub(r"n\in'", "ing", x))

# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())


exclude = set(['$','&','+',':',';','=','@','|','<','>','^','*','%','-','#','\'','।']) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

exclude = set(string.punctuation)
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))


# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines['hindi'] = lines['hindi'].apply(lambda x :  x + '\n')

lines['english'] = lines['english'].apply(lambda x : x + '\n')
# lines.head(20)
df=lines.copy()
df.head(20)

,english,hindi
0,give your application an accessibility workout\n,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें\n
1,accerciser accessibility explorer\n,एक्सेर्साइसर पहुंचनीयता अन्वेषक\n
2,the default plugin layout for the bottom panel\n,निचले पटल के लिए डिफोल्ट प्लगइन खाका\n
3,the default plugin layout for the top panel\n,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका\n
4,a list of plugins that are disabled by default\n,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...
5,highlight duration\n,अवधि को हाइलाइट रकें\n
6,the duration of the highlight box when selecti...,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से ...
7,highlight border color\n,सीमांत बोर्डर के रंग को हाइलाइट करें\n
8,the color and opacity of the highlight border\n,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता\n
9,highlight fill color\n,भराई के रंग को हाइलाइट करें\n


In [ ]:
from sklearn.model_selection import train_test_split
train_temp, test_data_sent = train_test_split(df, test_size = 0.2,random_state=42)
train_data_sent, validation_data_sent = train_test_split(train_temp, test_size=0.125,random_state=42)
print(test_data_sent)

                                             english                                             hindi
119737                                     a l m p\n                                         a l m p\n
72272                                      created\n                                          सृजितः\n
158154  writing clipboard data to temp file failed\n  क्लिपबोर्ड डाटा को अस्थाई फ़ाइल में लिखना असफल\n
65426                      use image as background\n       छवि पृष्ठ भूमि के रूप में उपयोग में लें u\n
30074            save project as a plain text list\n              बतौर सादा पाठ सूची परियोजना सहेजें\n
...                                              ...                                               ...
4174                          klondike three decks\n                              क्लॉन्डिके तीन डेक\n
91537                                africa malabo\n                                    अफ्रीकामलाबो\n
156449                             project options\n                     

In [ ]:
train_data_sent.to_json('train_data.json', orient='records', lines=True)
validation_data_sent.to_json('validation_data.json', orient='records', lines=True)
test_data_sent.to_json('test_data.json', orient='records', lines=True)

In [ ]:
from indicnlp.tokenize import indic_tokenize


def hindi_tokenizer(sentence):
    # sentence = re.sub(r"([.!?])", r" \1", sentence)
    # return [word for word in indic_tokenize.trivial_tokenize(normalizer.normalize(sentence.strip().split("-")[-1].strip()))]
    return [word for word in indic_tokenize.trivial_tokenize(sentence.strip().split("-")[-1].strip())]

In [ ]:
import spacy
en_tokenizer = spacy.load('en')

def english_tokenizer(sentence):
    # sentence = re.sub(r"([.!?])", r" \1", sentence)
    # sentence = re.sub(r"[^a-zA-Z.!?]+", r" ", sentence)
    return [word.text for word in en_tokenizer.tokenizer(sentence.strip().split("-")[-1].strip())][::-1]

In [ ]:
ENGLISH = Field(tokenize = english_tokenizer, init_token = '<sos>', eos_token = '<eos>', lower = True)
HINDI = Field(tokenize = hindi_tokenizer, init_token = '<sos>', eos_token = '<eos>')
fields = {'english': ('english', ENGLISH), 'hindi': ('hindi', HINDI)}

In [ ]:
train_data, validation_data, test_data = data.TabularDataset.splits(path = '',
                                        train = 'train_data.json',
                                        validation = 'validation_data.json',
                                        test = 'test_data.json',
                                        format = 'json',
                                        fields = fields)

print(len(train_data), len(validation_data), len(test_data))

140000 20000 40000


In [ ]:
# print(train_data_sent)
print(vars(train_data.examples[0]))

{'english': ['contact', 'given', 'the', 'with', 'conversation', 'a', 'start', 'not', 'could'], 'hindi': ['दिए', 'गए', 'संपर्क', 'के', 'साथ', 'वार्तालाप', 'नहीं', 'शुरू', 'कर', 'सकता', 'है']}


In [ ]:
ENGLISH.build_vocab(train_data, min_freq=2)
HINDI.build_vocab(train_data, min_freq=2)

print(f"English: {len(ENGLISH.vocab)}, Hindi: {len(HINDI.vocab)}")

English: 9825, Hindi: 14026


In [ ]:
import random


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, p):
        super().__init__()
        self.dropout = nn.Dropout(p)
        self.word_embeddings = nn.Embedding(input_size, embedding_size) 
        self.gru = nn.GRU(embedding_size, hidden_size, bidirectional=True)
        self.linear = nn.Linear(2 * hidden_size, hidden_size)
        self.hidden_size = hidden_size
        
    def forward(self, input):
        embeddings = self.dropout(self.word_embeddings(input))
        o, h = self.gru(embeddings) 
        h = torch.tanh(self.linear(torch.cat((h[-2,:,:], h[-1,:,:]), dim = 1)))

        return o, h

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        
        self.attention_layer = nn.Linear(3 * hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, h, eo):
        h = h.unsqueeze(1).repeat(1, eo.shape[0], 1)
        eo = eo.permute(1, 0, 2)
        e = torch.tanh(self.attention_layer(torch.cat((h, eo), dim = 2))) 
        alpha = self.linear(e).squeeze(2)
        
        return F.softmax(alpha, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, p, attention):
        super().__init__()
        self.dropout = nn.Dropout(p)
        self.word_embeddings = nn.Embedding(output_size, embedding_size)
        self.gru = nn.GRU(embedding_size + 2 * hidden_size, hidden_size)
        self.linear = nn.Linear(embedding_size + 3 * hidden_size, output_size)
        self.output_size = output_size
        self.attention = attention
        self.hidden_size = hidden_size

    def forward(self, input, h, eo):        
        embeddings = self.dropout(self.word_embeddings(input.unsqueeze(0)))
        alpha = self.attention(h, eo).unsqueeze(1)
        eo = eo.permute(1, 0, 2)
        w = torch.bmm(alpha, eo).permute(1, 0, 2)
        o, h = self.gru(torch.cat((embeddings, w), dim = 2), h.unsqueeze(0))
        predictions = self.linear(torch.cat((o, w, embeddings), dim = 2).squeeze(0))

        return predictions, h.squeeze(0),alpha.squeeze(1)

In [ ]:
class Model(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.device = device
        self.encoder = encoder
        self.decoder = decoder
         
    def forward(self, input, actual):
        
        eo, h = self.encoder(input)
        input = actual[0, :]
        predictions = torch.zeros(actual.shape[0], actual.shape[1], self.decoder.output_size).to(self.device)

        for t in range(1, actual.shape[0]):
            o, h,_ = self.decoder(input, h, eo)
            predictions[t] = o
            teacher_force=random.random() < 0.5

            predicted = o.argmax(1) 
            input = actual[t] if teacher_force else predicted

        return predictions      

In [ ]:
def train(model, train_data_iterator, optimizer, criterion):
    
    model.train()    
    total_loss = 0
    
    for batch in tqdm(train_data_iterator):
        input = batch.english
        actual = batch.hindi
        optimizer.zero_grad()
        predictions = model(input, actual)
        vocab_size = predictions.shape[-1]
        predictions = predictions[1:].view(-1, vocab_size)
        actual = actual[1:].view(-1) # flattening the sentence_length x batch_size dimensions because cross entropy loss needs 1d shape and also removing the sos token
        loss = criterion(predictions, actual)
        loss.backward()   
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        
    average_loss = total_loss / len(train_data_iterator)
    return average_loss

In [ ]:
def evaluate(model, data_iterator, criterion):
    
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(data_iterator):
            input = batch.english
            actual = batch.hindi
            predictions = model(input, actual)
            vocab_size = predictions.shape[-1]
            predictions = predictions[1:].view(-1, vocab_size)
            actual = actual[1:].view(-1) # flattening the sentence_length x batch_size dimensions because cross entropy loss needs 1d shape and also removing the sos token
            loss = criterion(predictions, actual)
            total_loss += loss.item()
    
    average_loss = total_loss / len(data_iterator)
    return average_loss

In [ ]:
enc = Encoder(len(ENGLISH.vocab), 200, 256, 0.5)
attention = Attention(256)
dec = Decoder(len(HINDI.vocab), 200, 256, 0.5, attention)
model = Model(enc, dec, device).to(device)

for name, param in model.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
model

Model(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (word_embeddings): Embedding(9825, 200)
    (gru): GRU(200, 256, bidirectional=True)
    (linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (word_embeddings): Embedding(14026, 200)
    (gru): GRU(712, 256)
    (linear): Linear(in_features=968, out_features=14026, bias=True)
    (attention): Attention(
      (attention_layer): Linear(in_features=768, out_features=256, bias=True)
      (linear): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = HINDI.vocab.stoi[HINDI.pad_token])
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, validation_data, test_data), batch_size = 32, device = device, sort = False)
train_data_iterator, validation_data_iterator, test_data_iterator = BucketIterator.splits((train_data, validation_data, test_data), batch_size = 32, device = device, sort = False)


In [ ]:
best_loss = 1e9


training_losses = []
validation_losses = []

for epoch in tqdm(range(10)):
    path="model "+str(epoch)+".pt"
    train_loss = train(model, train_data_iterator, optimizer, criterion)
    valid_loss = evaluate(model, validation_data_iterator, criterion)
    torch.cuda.empty_cache()
    training_losses.append(np.exp(train_loss))
    validation_losses.append(np.exp(valid_loss))
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(),'modelpunrev_200k.pt')
    
    print(f"Epoch: {epoch + 1}")
    print(f"Train Loss (exponent to analyse better): {np.exp(train_loss):.3f}")
    print(f"Val. Loss (exponent to analyse better): {np.exp(valid_loss):.3f}")


Epoch: 1
Train Loss (exponent to analyse better): 56.076
Val. Loss (exponent to analyse better): 11.244



Epoch: 2
Train Loss (exponent to analyse better): 7.907
Val. Loss (exponent to analyse better): 5.723


In [ ]:
import matplotlib.pyplot as plt

plt.plot(training_losses, label = 'Training')
plt.plot(validation_losses, label = 'Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# !cp 'drive/MyDrive/NLP_project/model1_200000.pt' .

In [ ]:
train_data_iterator, validation_data_iterator, test_data_iterator = BucketIterator.splits((train_data, validation_data, test_data), batch_size = 24, device = device, sort = False)

model.load_state_dict(torch.load('modelpunrev_200k.pt'))
test_loss = evaluate(model, test_data_iterator, criterion)
print(f"Test Loss (exponent to analyse better): {np.exp(test_loss):.3f}")

In [ ]:
def translate(text,max_len=50):
  model.eval()
  tokens = english_tokenizer(text)
  tokens = [ENGLISH.init_token] + tokens + [ENGLISH.eos_token]
  src_indexes = [ENGLISH.vocab.stoi[token] for token in tokens]
  src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
  src_tensor = src_tensor.reshape(-1,1)
  with torch.no_grad():
      encoder_outputs, hidden = model.encoder(src_tensor)
  trg_indexes = [HINDI.vocab.stoi[HINDI.init_token]]

  attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
  for i in range(max_len):

      trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
              
      with torch.no_grad():
          output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs)

      attentions[i] = attention
          
      pred_token = output.argmax(1).item()
      
      trg_indexes.append(pred_token)

      if pred_token == HINDI.vocab.stoi[HINDI.eos_token]:
          break
  
  trg_tokens = [HINDI.vocab.itos[i] for i in trg_indexes]
  
  return trg_tokens[1:], attentions[:len(trg_tokens)-1]

print(translate("man speaking native language:"))

In [ ]:

file1 = open("alignmentspun_300000.txt","w")#write mode 

co=0
for i, (eng_sentence, hin_sentence) in enumerate(zip(test_data_sent["english"], test_data_sent["hindi"])):

  hindi_predicted ,attention = translate(eng_sentence)
  hindlen=hindi_predicted
  tokens = english_tokenizer(eng_sentence)
  tok=english_tokenizer(eng_sentence)
  if len(tok)>=3:
    print(hindi_predicted)
    print(tokens)
  
    hindi_predicted = " ".join(list(filter(lambda x: x != '<eos>', hindi_predicted))).strip()
    tokens=" ".join(list(filter(lambda x: x != '<eos>', tokens))).strip()
    input_sentence="0 ||| "
    input_sentence=input_sentence + (hindi_predicted).strip() + " ||| 0 ||| "
    input_sentence+=" ".join([ENGLISH.init_token])
    input_sentence=input_sentence + " " + (tokens).strip() + " ||| " + str(len(tok)+2) + " " + str(len(hindlen))
    file1.write(input_sentence)
    file1.write("\n")

    attention = attention.squeeze(1).cpu().detach().numpy()
    input_mat=""

    for x in attention:
      input_mat=""
      for y in x:
        input_mat+=str(y)+ " "
      file1.write(input_mat)
      file1.write("\n")    
    file1.write("\n")
    co+=1
    if(co>=10):
      break


file1.close()

In [ ]:
def translate1(text):
  model.eval()
  with torch.no_grad():
    tokens = english_tokenizer(text)
    tokens = [ENGLISH.init_token] + tokens + [ENGLISH.eos_token]
    src_indexes = [ENGLISH.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_tensor = src_tensor.reshape(-1,1)

    output = model(src_tensor, src_tensor)
    output_dim = output.shape[-1]
    output = output.view(-1, output_dim)
    indices = torch.argmax(output,dim=1).tolist()
    return [HINDI.vocab.itos[x] for x in indices]

print(translate1("man speaking native language:"))

In [ ]:


import nltk

bleu_i = []
bleu_o = []

for i, (eng_sentence, hin_sentence) in enumerate(zip(test_data_sent["english"], test_data_sent["hindi"])):

  hindi_predicted = translate1(eng_sentence)
  hindi_predicted = " ".join(list(filter(lambda x: x != '<eos>', hindi_predicted))[1:]).strip()
  # csvwriter.writerow([eng_sentence.strip(), hindi_predicted.strip(), hin_sentence.strip()])
  if len(hin_sentence.strip())>1 and len(hindi_predicted.strip()) >1 : 
    bleu_i.append(hin_sentence.strip())
    bleu_o.append(hindi_predicted.strip())


# translated_sentences.close()

BLEU_scores = []

for i in range(len(bleu_i)):
  # print(bleu_i[i],len(bleu_i[i]))
  # print(bleu_o[i],len(bleu_o[i]))

  BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu(bleu_i[i], bleu_o[i],smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))
  
print("Average BLEU Score:", np.mean(BLEU_scores))

In [ ]:
!cp './modelpunrev_200k.pt' -r 'drive/MyDrive/NLP_project/modelpunrev_200000.pt'

In [ ]:
!cp './alignmentspun_300000.txt' -r 'drive/MyDrive/NLP_project/algnpunrev_200000.txt'